# Intelligent Agents: Reflex-Based Agents for the Vacuum-cleaner World


## Instructions

Total Points: Undergrads 100 / Graduate students 110

Complete this notebook. Use the provided notebook cells and insert additional code and markdown cells as needed. Submit the completely rendered notebook as a PDF file. 

## Introduction

In this assignment you will implement a simulator environment for an automatic vacuum cleaner robot, a set of different reflex-based agent programs, and perform a comparison study for cleaning a single room. Focus on the __cleaning phase__ which starts when the robot is activated and ends when the last dirty square in the room has been cleaned. Someone else will take care of the agent program needed to navigate back to the charging station after the room is clean.

## PEAS description of the cleaning phase

__Performance Measure:__ Each action costs 1 energy unit. The performance is measured as the sum of the energy units used to clean the whole room.

__Environment:__ A room with $n \times n$ squares where $n = 5$. Dirt is randomly placed on each square with probability $p = 0.2$. For simplicity, you can assume that the agent knows the size and the layout of the room (i.e., it knows $n$). To start, the agent is placed on a random square.

__Actuators:__ The agent can clean the current square (action `suck`) or move to an adjacent square by going `north`, `east`, `south`, or `west`.

__Sensors:__ Four bumper sensors, one for north, east, south, and west; a dirt sensor reporting dirt in the current square.  


## The agent program for a simple randomized agent

The agent program is a function that gets sensor information (the current percepts) as the arguments. The arguments are:

* A dictionary with boolean entries for the for bumper sensors `north`, `east`, `west`, `south`. E.g., if the agent is on the north-west corner, `bumpers` will be `{"north" : True, "east" : False, "south" : False, "west" : True}`.
* The dirt sensor produces a boolean.

The agent returns the chosen action as a string.

Here is an example implementation for the agent program of a simple randomized agent:  

In [16]:
import numpy as np

actions = ["north", "east", "west", "south", "suck"]

def simple_randomized_agent(bumpers, dirty):
    return np.random.choice(actions)

In [17]:
# define percepts (current location is NW corner and it is dirty)
bumpers = {"north" : True, "east" : False, "south" : False, "west" : True}
dirty = True

# call agent program function with percepts and it returns an action
simple_randomized_agent(bumpers, dirty)

'west'

__Note:__ This is not a rational intelligent agent. It ignores its sensors and may bump into a wall repeatedly or not clean a dirty square. You will be asked to implement rational agents below.

## Simple environment example

We implement a simple simulation environment that supplies the agent with its percepts.
The simple environment is infinite in size (bumpers are always `False`) and every square is always dirty, even if the agent cleans it. The environment function returns a performance measure which is here the number of cleaned squares (since the room is infinite and all squares are constantly dirty, the agent can never clean the whole room as required in the PEAS description above). The energy budget of the agent is specified as `max_steps`. 

In [18]:
def simple_environment(agent, max_steps, verbose = True):
    num_cleaned = 0
    
    for i in range(max_steps):
        dirty = True
        bumpers = {"north" : False, "south" : False, "west" : False, "east" : False}

        action = agent(bumpers, dirty)
        if (verbose): print("step", i , "- action:", action) 
        
        if (action == "suck"): 
            num_cleaned = num_cleaned + 1
        
    return num_cleaned
        


Do one simulation run with a simple randomized agent that has enough energy for 20 steps.

In [19]:
simple_environment(simple_randomized_agent, max_steps = 20)

step 0 - action: north
step 1 - action: west
step 2 - action: north
step 3 - action: suck
step 4 - action: east
step 5 - action: east
step 6 - action: west
step 7 - action: suck
step 8 - action: east
step 9 - action: south
step 10 - action: south
step 11 - action: north
step 12 - action: east
step 13 - action: east
step 14 - action: suck
step 15 - action: west
step 16 - action: east
step 17 - action: east
step 18 - action: north
step 19 - action: west


3

# Tasks

## General [10 Points]

1. Make sure that you use the latest version of this notebook. Sync your forked repository and pull the latest revision. 
2. Your implementation can use libraries like math, numpy, scipy, but not libraries that implement inteligent agents or complete search algorithms. Try to keep the code simple! In this course, we want to learn about the algorithms and we often do not need to use object-oriented design.
3. You notebook needs to be formated professionally. 
    - Add additional markdown blocks for your description, comments in the code, add tables and use mathplotlib to produce charts where appropriate
    - Do not show debugging output or include an excessive amount of output.
    - Check that your PDF file is readable. For example, long lines are cut off in the PDF file. You don't have control over page breaks, so do not worry about these.
4. Document your code. Add a short discussion of how your implementation works and your design choices.


## Task 1: Implement a simulation environment [20 Points]

The simple environment above is not very realistic. Your environment simulator needs to follow the PEAS description from above. It needs to:

* Initialize the environment by storing the state of each square (clean/dirty) and making some dirty. ([Help with random numbers and arrays in Python](https://github.com/mhahsler/CS7320-AI/blob/master/Python_Code_Examples/random_numbers_and_arrays.ipynb))
* Keep track of the agent's position.
* Call the agent function repeatedly and provide the agent function with the sensor inputs.  
* React to the agent's actions. E.g, by removing dirt from a square or moving the agent around unless there is a wall in the way.
* Keep track of the performance measure. That is, track the agent's actions until all dirty squares are clean and count the number of actions it takes the agent to complete the task.

The easiest implementation for the environment is to hold an 2-dimensional array to represent if squares are clean or dirty and to call the agent function in a loop until all squares are clean or a predefined number of steps have been reached (i.e., the robot runs out of energy).

The simulation environment should be a function like the `simple_environment()` and needs to work with the simple randomized agent program from above. **Use the same environmnt for all your agent implementations in the tasks below.**

*Note on debugging:* Debugging is difficult. Make sure your environment prints enough information when you use `verbose = True`. Also, implementing a function that the environment can use to displays the room with dirt and the current position of the robot at every step is very useful.  

In [24]:
# Your code and description goes here
'''
    My code created a 2d array of squres with a percent chance that the square might be dirty 
    from there I run the enviroment on a pre-define counter for steps taken or until the
    enviroment is cleaned
'''
def mysimple_enviroment(agent, verbose, xsize, ysize):
    #define max steps that robot can take
    max_steps = int((xsize * ysize) * 1.2 * 1.5)
    
    #random generaator
    import numpy as np
    rng = np.random.default_rng()
    p = .2

    #how big the enviroment is
    rows, cols = (xsize, ysize)
    
    #randomize the location
    locx = np.random.randint(4)
    locy = np.random.randint(4)
    
    #number of dirty squres
    num_dirty = 0
    
    #create enviroment
    #if squre is clean it's a zero if it is dirty it is a 1
    arr=[]
    for i in range(cols):
        col = []
        for j in range(rows):
            if rng.random() > p:
                #clean
                col.append(0)
            else:
                #dirty
                col.append(1)
                num_dirty = num_dirty + 1 
        arr.append(col)
    
    #print enviroment before any actions have been taken
    if (verbose):
        for i in range(rows):
            print(arr[i])
    
   
    num_cleaned =0;
    for i in range(max_steps):
        #print current location and if the current squre is dirty
        if (verbose):
            print(f"locx:{locx}, locy:{locy}")
            print(f"loc is dirty:{arr[locx][locy]}")
        #store the current square
        dirty = arr[locx][locy]
        #define the sensors
        bumpers = {"north" : False, "south" : False, "west" : False, "east" : False}
        #if the location of the robot is next to a wall then the sensor is true
        if locx == 0:
            bumpers["north"] = True
        if locy == 0:
            bumpers["west"] = True
        if locx == cols-1:
            bumpers["south"] = True
        if locy == rows-1:
            bumpers["east"] = True
            
        #call the model based reflex agent and pass in if is the square is dirty 
        #and the bumpers
        #action = model_based_reflex_agent(bumpers, dirty)
        action = agent(bumpers, dirty)
        #if the action is suck clean the aqure
        if action == "suck":
            arr[locx][locy] = 0
            num_dirty = num_dirty - 1 
            #print the new enviorment
            if (verbose):
                for i in range(rows):
                    print(arr[i])
        #if the action is north move up
        elif action == "north":
            locx = locx - 1
        #if action is south move down
        elif action == "south":
            locx = locx + 1
        #if action is east move right
        elif action == "east":
            locy = locy + 1
        #if action is west move left
        elif action == "west":
            locy = locy - 1
        ##if action is end the room is cleaned 
        elif action == "end":
            return num_cleaned
        if (verbose): print("step", i , "- action:", action) 
        
        if (action == "suck"): 
            num_cleaned = num_cleaned + 1
        
    return num_cleaned
"""
is_leftcor = False
is_leftwall = False
direction = "east"
global is_leftcor
global is_leftwall
global direction
mysimple_enviroment(model_based_reflex_agent, verbose = True, xsize = 5, ysize = 5)
"""

[0, 0, 1, 0, 1]
[0, 0, 0, 0, 0]
[0, 0, 0, 1, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 1]
locx:3, locy:0
loc is dirty:0
step 0 - action: north
locx:2, locy:0
loc is dirty:0
step 1 - action: north
locx:1, locy:0
loc is dirty:0
step 2 - action: north
locx:0, locy:0
loc is dirty:0
step 3 - action: None
locx:0, locy:0
loc is dirty:0
step 4 - action: east
locx:0, locy:1
loc is dirty:0
step 5 - action: east
locx:0, locy:2
loc is dirty:1
[0, 0, 0, 0, 1]
[0, 0, 0, 0, 0]
[0, 0, 0, 1, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 1]
step 4 - action: suck
locx:0, locy:2
loc is dirty:0
step 7 - action: east
locx:0, locy:3
loc is dirty:0
step 8 - action: east
locx:0, locy:4
loc is dirty:1
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 1, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 1]
step 4 - action: suck
locx:0, locy:4
loc is dirty:0
step 10 - action: south
locx:1, locy:4
loc is dirty:0
step 11 - action: west
locx:1, locy:3
loc is dirty:0
step 12 - action: west
locx:1, locy:2
loc is dirty:0
step 13 - action: west
locx:1, locy:1
loc is di

4

## Task 2:  Implement a simple reflex agent [10 Points] 

The simple reflex agent randomly walks around but reacts to the bumper sensor by not bumping into the wall and to dirt with sucking. Implement the agent program as a function.

_Note:_ Agents cannot directly use variable in the environment. They only gets the percepts as the arguments to the agent function.

In [21]:
# Your code and description goes here
import numpy as np

#actions = ["north", "east", "west", "south", "suck"]

#this agent reacts to the enviroment
def simple_reflex_agent(bumpers, dirty):
    # if the squre is dirty return suck which cleans the squre
    if dirty == 1:
        return "suck"
    #if the north bumper is true and the west bumper is true 
    elif(bumpers["north"]== True and bumpers["west"] == True):
        #then the only place it can go is east and south randomly pick one
        actions=["east","south"]
        return np.random.choice(actions)
    #if the north bumper is true and the east bumper is true 
    elif(bumpers["north"]== True and bumpers["east"] == True):
        #then the only place it can go is west and south randomly pick one
        actions=["west","south"]
        return np.random.choice(actions)
    #if the south bumper is true and the east bumper is true 
    elif(bumpers["south"]== True and bumpers["east"] == True):
        #then the only place it can go is west and north randomly pick one
        actions=["west","north"]
        return np.random.choice(actions)
    #if the south bumper is true and the west bumper is true 
    elif(bumpers["south"]== True and bumpers["west"] == True):
        #then the only place it can go is east and north randomly pick one
        actions=["east","north"]
        return np.random.choice(actions)
    #if the north bumper is true  
    elif(bumpers["north"] == True):
        #then the only place it can go is east, south and west randomly pick one
        actions=["east","south", "west"]
        return np.random.choice(actions)
    #if the south bumper is true  
    elif(bumpers["south"] == True):
        #then the only place it can go is east, north and west randomly pick one
        actions=["east","north", "west"]
        return np.random.choice(actions)
    #if the east bumper is true  
    elif(bumpers["east"] == True):
        #then the only place it can go is south, north and west randomly pick one
        actions=["south","north", "west"]
        return np.random.choice(actions)
    #if the west bumper is true  
    elif(bumpers["west"] == True):
        #then the only place it can go is south, north and east randomly pick one
        actions=["south","north", "east"]
        return np.random.choice(actions)
    #if none of the bumpers are ture randomly pick a direction
    else:
        actions = ["north", "east", "west", "south"]
        return np.random.choice(actions)

## Task 3: Implement a model-based reflex agent [20 Points]

Model-based agents use a state to keep track of what they have done and perceived so far. Your agent needs to find out where it is located and then keep track of its current location. You also need a set of rules based on the state and the percepts to make sure that the agent will clean the whole room. For example, the agent can move to a corner to determine its location and then it can navigate through the whole room and clean dirty squares.

Describe how you define the __agent state__ and how your agent works before implementing it. ([Help with implementing state information on Python](https://github.com/mhahsler/CS7320-AI/blob/master/Python_Code_Examples/store_agent_state_information.ipynb))

In [22]:
# Your short description of the state and your implementation goes here
""" My code moves the robot to the left wall first then moves the robot all the way north which is the top 
left cornor of the room. Once my robot reaches the left cornor it was jjust cleaning the room by going 
all the way east then going the the way down one and then all the west until it reaches the end.
"""

' My code moves the robot to the left wall first then moves the robot all the way north which is the top \nleft cornor of the room. Once my robot reaches the left cornor it was jjust cleaning the room by going \nall the way east then going the the way down one and then all the west until it reaches the end.\n'

In [14]:
# Your code goes here
#defined variable states
is_leftcor = False
is_leftwall = False
direction = "east"
def model_based_reflex_agent(bumpers, dirty):
    #defined global variables
    global is_leftcor
    global is_leftwall
    global direction
    #if the robot hasn't reached the left cornor yet
    if is_leftcor == False:
        #if the robot has reached the left wall
        if bumpers["west"] == True:
            #set left wall to true
            is_leftwall = True 
        #check bumpers to see if the robot has reached the left cornor
        if bumpers["west"] == True and bumpers["north"] == True:
            #set left cornor to true
            is_leftcor = True
        #if the left cornor is false
        if is_leftcor == False:
            #if the left wall is false
            if is_leftwall == False:
                #move west until the robot finds left wall
                return "west"
            else:
                #then move north
                return "north"
    #once the robot has found the left cornor
    else:
        #check to see if the square is dirty if it is clean it
        if dirty == 1:
            return "suck"
        #if the direction is heading has the bumper set to true
        elif bumpers[direction] == True:
            #check to see if it can move down
            if bumpers["south"] == True:
                #if it can't then the robot has cleaned the room
                return "end"
            #if it can move down check the direction it was heading if it was east chnage direction to west
            if direction == "east":
                direction = "west"
            else:
                #else go east
                direction = "east"
            #it will always move south
            return "south"
        #else go in the direction it was heading
        else:
            return direction
           

## Task 4: Simulation study [30 Points]

Compare the performance (the performance measure is defined in the PEAS description above) of the agents using  environments of different size. E.g., $5 \times 5$, $10 \times 10$ and
$100 \times 100$. Use 100 random runs for each. Present the results using tables and graphs. Discuss the differences between the agents. 
([Help with charts and tables in Python](https://github.com/mhahsler/CS7320-AI/blob/master/Python_Code_Examples/charts_and_tables.ipynb))

In [26]:
# Your code goes here
import numpy as np  
import pandas as pd

# create data for n = 10 and N = 100 replications, so we can compare distributions.
N = 100
performance = {
    "agent 1" : np.add(mysimple_enviroment(simple_reflex_agent, verbose, xsize, ysize)),
    "agent 2" : np.add(mysimple_enviroment(simple_randomized_agent, verbose, xsize, ysize)),
    "agent 3" : np.add(
        is_leftcor = False,
        is_leftwall = False,
        direction = "east",
        global is_leftcor,
        global is_leftwall,
        global direction,
        mysimple_enviroment(model_based_reflex_agent, verbose, xsize, ysize))
}

# show data as a data frame using pandas
df = pd.DataFrame(performance)
df


SyntaxError: invalid syntax (3752734981.py, line 14)

Fill out the following table with the average performance measure for 100 random runs (you may also create this table with code):

| Size     | Randomized Agent | Simple Reflex Agent | Model-based Reflex Agent |
|----------|------------------|---------------------|--------------------------|
| 5x5     | | | |
| 10x10   | | | |
| 100x100 | | | |

Add charts to compare the performance of the different agents.

In [10]:
# Your graphs and discussion of the results goes here

## Task 5: Robustness of the agent implementations [10 Points] 

Describe how **your agent implementations** will perform 

* if it is put into a rectangular room with unknown size, 
* if the cleaning area can have an iregular shape (e.g., a hallway connecting two rooms), or 
* if the room contains obstacles (i.e., squares that it cannot pass through and trigger the bumper sensors).

In [11]:
# Answer goes here

## Graduate student advanced task: Obstacles [10 Points]

__Undergraduate students:__ This is a bonus task you can attempt if you like [+5 Bonus Points].

1. Change your simulation environment tor run experiments for the following problem: Add random obstacle squares that also trigger the bumper sensor. The agent does not know where the obstacles are. Observe how this changes the performance of the three implementations.

2. Describe what would need to be done to perform better with obstacles. Add code if you can. 

In [12]:
# Your code and discussion goes here

## More advanced implementation tasks

* __Agent for and environment with obstacles:__ Implement an agent for an environment where the agent does not know how large the environment is (we assume it is rectangular), where it starts or where the obstacles are. An option would be to always move to the closest unchecked/uncleaned square (note that this is actualy depth-first search).

* __Utility-based agent:__ Change the environment for a $5 \times 5$ room, so each square has a fixed probability of getting dirty again. For the implementation, we give the environment a 2-dimensional array of probabilities. The utility of a state is defined as the number of currebntly clean squares in the room. Implement a utility-based agent that maximizes the expected utility over one full charge which lasts for 100000 time steps. To do this, the agent needs to learn the probabilities with which different squares get dirty again. This is very tricky!

In [13]:
# Your ideas/code